# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686535


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:11,  2.57s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:55,  2.06s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:38,  1.49s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:27,  1.11s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:00,  7.80it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  7.49it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  7.97it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  8.33it/s]

not-logged-in-c3ca171c6551a9bcc607      0.001452
shirogane                               0.000639
richiemac                               0.000527
not-logged-in-a9247261537fa1da33fe    137.404685
cschwefl                                0.060382
ctiborekskutr                           0.000781
CThomas                                 0.004020
Camelron                                0.000867
thedarktree                             0.002477
not-logged-in-762016fa77f900b47089      0.001578
not-logged-in-5f30b2bb9b7f9d49da41      0.032178
BirdLeaf                                0.075577
goldenaj1980                            0.002263
not-logged-in-095f56945581547baa6a      0.001543
Lavadude                                0.010216
clars915                                0.000677
Peyton.Harvey                           0.117623
cooper.hull                             0.002463
not-logged-in-9f43813670b3af3a0ed7      0.002913
kgenere                                 0.060013
ywan3529            

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())